# Eyeriss

This notebook reproduces the salient characteristics of the [Eyeriss](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7738524) accelerator.

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src import utils

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below.

**Warning:** Large tensors will overwhelm the video generation. Either:
1. Use small tensors; as a rule of thumb, fewer than 60 computes (e.g., multiplications) should be required.
2. Do not generate a video; remove the `spacetime` specification from the `mapping` before compiling.

In [ ]:
# Filter
M = 2
C = 2
R = 2
S = 2

# Input
N = 2
H = 3
W = 3

# Stride
Stride = 1

# Output
E = int((H-R+Stride)/Stride)
F = int((W-S+Stride)/Stride)

# Partition parameters
N1 = 2
N0 = 1
C2 = 2
C1 = 2
C0 = 2
M2 = 2
M1 = 1
M0 = 1
E2 = 2
E1 = 2
E0 = 2

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

## Compile and Run

Below is the TeAAL specification for Eyeriss. To simulate the accelerator:
1. Compile it to HiFiber by running the cell, inserting a new cell
2. Run the new cell, which will
    - Execute the kernel; multiplying the above defined matrices
    - Generate visualizations of the actions of the kernel
 
   

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(M1), uniform_shape(M0)]
      N: [uniform_shape(N0)]
      C: [uniform_shape(C1), uniform_shape(C0)]
      E: [uniform_shape(E1), uniform_shape(E0)]
  loop-order:
    O: [N1, C2, M2, C1, M1, N0, C0, M0, F, E2, E1, E0, R, S]
  spacetime:
    O:
      space: [C1, M1, E1, E0, R]
      time: [N1, C2, M2, N0, C0, M0, F, E2, S]
"""
utils.compile(yaml, generate_video = True)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

## Extra specifications for Eyeriss with AlexNet Convolution layers as the target layers to model

Original AlexNet assumes batch size = 128.
Eyeriss uses batch size = 4.
We follow the Eyeriss parameter here. 

Due to the large parameter involved in AlexNet, our spec will not generate videos (generate_video = False), but it still checks correctness. 

![Eyerisis AlexNet detail](./images/alexnet-eyeriss.png)

#### Warning

The below cells (Original Parameters) might take a long time to run. You can decrease the parameters (Reduced Parameters) to speed up the process.

### AlexNet Convolutional Layer 1

#### Original Parameters

In [ ]:
# Filter
M = 96
C = 3
R = 11
S = 11

# Input
N = 4
H = 227
W = 227

# Stride
Stride = 4

# Output
E = 55
F = 55

# Partition parameters
M2 = M
M1 = 32
M0 = 16
E2 = E 
E1 = 14
E0 = 7

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, 4*e+r, 4*f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(32), uniform_shape(16)]
      E: [uniform_shape(14), uniform_shape(7)]
  loop-order:
    O: [N, C, M2, M1, M0, F, E2, E1, E0, R, S]
  spacetime:
    O:
      space: [M1, E1, E0, R]
      time: [N, C, M2, M0, F, E2, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

#### Reduced Parameters

In [ ]:
# Filter
M = 16
C = 3
R = 4
S = 4

# Input
N = 4
H = 32
W = 12

# Stride
Stride = 4

# Output
E = 8 
F = 3

# Partition parameters
M2 = M
M1 = 8
M0 = 4
E2 = E 
E1 = 4
E0 = 2

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, 4*e+r, 4*f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(8), uniform_shape(4)]
      E: [uniform_shape(4), uniform_shape(2)]
  loop-order:
    O: [N, C, M2, M1, M0, F, E2, E1, E0, R, S]
  spacetime:
    O:
      space: [M1, E1, E0, R]
      time: [N, C, M2, M0, F, E2, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

### AlexNet Convolutional Layer 2

#### Original Parameters

In [ ]:
# Filter
M = 256
C = 48
R = 5
S = 5

# Input
N = 4
H = 31
W = 31

# Stride
Stride = 1

# Output
E = int((H-R+Stride)/Stride)
F = int((W-S+Stride)/Stride)

# Partition parameters
C1 = C
C0 = 2
M1 = M
M0 = 16

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(16)]
      C: [uniform_shape(2)]
  loop-order:
    O: [N, C1, M1, C0, M0, F, E, R, S]
  spacetime:
    O:
      space: [E, R]
      time: [N, C1, M1, C0, M0, F, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

#### Reduced Parameters

In [ ]:
# Filter
M = 8
C = 6
R = 5
S = 5

# Input
N = 4
H = 8
W = 8

# Stride
Stride = 1

# Output
E = int((H-R+Stride)/Stride)
F = int((W-S+Stride)/Stride)

# Partition parameters
C1 = C
C0 = 2
M1 = M
M0 = 4

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(4)]
      C: [uniform_shape(2)]
  loop-order:
    O: [N, C1, M1, C0, M0, F, E, R, S]
  spacetime:
    O:
      space: [E, R]
      time: [N, C1, M1, C0, M0, F, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

### AlexNet Convolutional Layer 3


#### Original Parameters

In [ ]:
# Filter
M = 384
C = 256
R = 3
S = 3

# Input
N = 4
H = 15
W = 15

# Stride
Stride = 1

# Output
E = 13
F = 13

# Partition parameters
C1 = C
C0 = 4
M2 = M
M1 = 64
M0 = 16
N1 = 4
N0 = 4

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(64), uniform_shape(16)]
      C: [uniform_shape(4)]
      N: [uniform_shape(4)]
  loop-order:
    O: [N1, C1, M2, M1, N0, C0, M0, F, E, R, S]
  spacetime:
    O:
      space: [M1, E, R]
      time: [N1, C1, M2, N0, C0, M0, F, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

#### Reduced Parameters

In [ ]:
# Filter
M = 8
C = 16
R = 3
S = 3

# Input
N = 4
H = 15
W = 15

# Stride
Stride = 1

# Output
E = 13
F = 13

# Partition parameters
C1 = C
C0 = 4
M2 = M
M1 = 4
M0 = 2
N1 = 4
N0 = 4

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(4), uniform_shape(2)]
      C: [uniform_shape(4)]
      N: [uniform_shape(4)]
  loop-order:
    O: [N1, C1, M2, M1, N0, C0, M0, F, E, R, S]
  spacetime:
    O:
      space: [M1, E, R]
      time: [N1, C1, M2, N0, C0, M0, F, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

### AlexNet Convolutional Layer 4 & 5


#### Original Parameters

In [ ]:
# Filter
M = 256
C = 192
R = 3
S = 3

# Input
N = 4
H = 15
W = 15

# Stride
Stride = 1

# Output
E = 13
F = 13

# Partition parameters
C2 = C
C1 = 6
C0 = 3
M2 = M
M1 = 32
M0 = 16
N1 = 4
N0 = 4

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(32), uniform_shape(16)]
      C: [uniform_shape(6), uniform_shape(3)]
      N: [uniform_shape(4)]
  loop-order:
    O: [N1, C2, M2, C1, M1, N0, C0, M0, F, E, R, S]
  spacetime:
    O:
      space: [C1, M1, E, R]
      time: [N1, C2, M2, N0, C0, M0, F, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)

#### Reduced Parameters

In [ ]:
# Filter
M = 6
C = 18
R = 3
S = 3

# Input
N = 4
H = 15
W = 15

# Stride
Stride = 1

# Output
E = 13
F = 13

# Partition parameters
C2 = C
C1 = 6
C0 = 3
M2 = M
M1 = 4
M0 = 2
N1 = 4
N0 = 4

# Random Input Tensors
I_NCHW = Tensor.fromRandom(rank_ids=["N", "C", "H", "W"], density=1.0, shape=[N, C, H, W])
F_MCRS = Tensor.fromRandom(rank_ids=["M", "C", "R", "S"], density=1.0, shape=[M, C, R, S])

In [ ]:
yaml = """
einsum:
  declaration:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  expressions:
    - O[n, m, e, f] = I[n, c, e+r, f+s]*F[m, c, r, s]
mapping:
  rank-order:
    I: [N, C, H, W]
    F: [M, C, R, S]
    O: [N, M, E, F]
  partitioning:
    O:
      M: [uniform_shape(4), uniform_shape(2)]
      C: [uniform_shape(6), uniform_shape(3)]
      N: [uniform_shape(4)]
  loop-order:
    O: [N1, C2, M2, C1, M1, N0, C0, M0, F, E, R, S]
  spacetime:
    O:
      space: [C1, M1, E, R]
      time: [N1, C2, M2, N0, C0, M0, F, S]
"""

utils.compile(yaml, generate_video = False)

In [ ]:
utils.check_conv(I_NCHW, F_MCRS, O_NMEF, stride=Stride)